Input: Hansen forest change script  
https://github.com/Revalue-Nature/notebooks-early-assessment/blob/main/Global_Hansen_ForestChange_v4.ipynb

In [12]:
%matplotlib inline
import tkinter as ttk
from tkinter import Tk, filedialog, simpledialog, messagebox
import os
import geopandas as gpd
import pandas as pd
import rasterio
from rasterio.plot import show
from rasterio.warp import reproject, Resampling, calculate_default_transform
import numpy as np
from osgeo import gdal

### Specify input variables

In [13]:
# Specify input name 
jurisdictionname = "Sofala"
jrd_column_name = "NAME_1"
projection = "EPSG:32737"

In [14]:
# Specify time period
t1 = 2015
t2 = 2021

In [15]:
root = Tk() # pointing root to Tk() to use it as Tk() in program.
root.withdraw() # Hides small tkinter window.
root.attributes('-topmost', True) # Opened windows will be active. above all windows despite of selection.
jrd_directory = filedialog.askopenfile(title="Open Jurisdiction Shapefile") # Returns opened path as st
# Raster folder
raster_directory = filedialog.askdirectory(title = "Specify Raster Folder Directory")
# Output folder
output_directory = filedialog.askdirectory(title = "Specify Output Folder")

2023-05-31 19:43:14.934 Python[9770:715543] +[CATransaction synchronize] called within transaction
2023-05-31 19:43:14.951 Python[9770:715543] +[CATransaction synchronize] called within transaction
2023-05-31 19:43:17.273 Python[9770:715543] +[CATransaction synchronize] called within transaction
2023-05-31 19:43:37.440 Python[9770:715543] +[CATransaction synchronize] called within transaction
2023-05-31 19:43:50.122 Python[9770:715543] +[CATransaction synchronize] called within transaction


In [16]:
# output directory
out_dir = os.path.expanduser(output_directory+'/data')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [17]:
# Jurisdiction
jrd = gpd.read_file(jrd_directory.name)

# Reproject Shapefile to UTM
jrd_proj  = jrd.to_crs(crs=projection)

### Raster data preparation

In [18]:
# Import Hansen forest change raster
fcc_path = os.path.join(raster_directory, jurisdictionname + '_ForestChange_' + str(t1) + '-' + str(t2) + '_jrd.tif').replace(os.sep, '/')
fcc_tif = rasterio.open(fcc_path)

In [19]:
np.unique(fcc_tif.read(1))

array([  0, 100, 116, 117, 118, 119, 120, 121], dtype=uint8)

In [35]:
# Reclassify raster pixel value
# 116-118 = 1
# 119-121 = 2
# 100 = 3

with rasterio.open(fcc_path) as src:    
    # Read as numpy array
    array = src.read()
    profile = src.profile
    profile.update(compress='lzw')

    # Reclassify
    array[((t1 - 2000 +100) < array) & (array <= (t1 - 2000 + 100 + 3))] = 1
    array[((t1 - 2000 + 100 + 3) < array) & (array <= (t2 - 2000 +100))] = 2
    array[array == 100] = 3
    # and so on ...  

with rasterio.open(os.path.join(out_dir, jurisdictionname + '_fcc123.tif').replace(os.sep, '/'), 'w', **profile) as dst:
    # Write to disk
    dst.write(array)